# Simple ML - Testing simple models#

What it does?
- Reads the csv file created by the Statistics Parser
- Drops the string columns and keeps only the columns with the grades
- Tests a model for each course-column using the LeaveOneOut method and the XGBoost Regressor
- Prints the results in terms of MAE and RMSE

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("csd.csv")
display(data)

,Ηλικία,Φύλο,Επέλεξα τη σχολή μου διότι:,"Κατά μέσο όρο την εβδομάδα, διαβάζω:","Μέσα στο εξάμηνο, παρακαλουθώ:",Υπήρξε ανάγκη για φροντηστηριακή βοήθεια σε κάποιο μάθημα έως τώρα;,"Μετά το πτυχίο, θα ήθελα να ακολουθήσω:",Ποιο από τα παρακάτω ισχύει;,Η σχολή απέχει από το σπίτι μου:,Ασχολούμαι εβδομαδιαία με:,...,NDM-06-02,NDM-06-04,NIS-07-08,NDM-07-01,NDM-07-02,NDM-08-02,NGE-08-02,NIS-07-06,NDM-08-03,NIS-08-04
0,19,Κορίτσι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,0 - 2 ώρες,Όλες τις διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Ζω με τους γονείς μου (ή γενικότερα την οικογέ...,25 - 45 λεπτά,Σειρές / Ταινίες,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,19,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,> 5 ώρες,Παραπάνω από τις μισές διαλέξεις,Όχι,Διδακτορικές Σπουδές,Μένω μόνος/-η μου,< 10 λεπτά,"Αθλητισμό, Video Games, Σειρές / Ταινίες",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,22,Κορίτσι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,2 - 5 ώρες,Παραπάνω από τις μισές διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Ζω με τους γονείς μου (ή γενικότερα την οικογέ...,25 - 45 λεπτά,"Αθλητισμό, Σειρές / Ταινίες, Εθελοντισμός",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,20,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο.,> 5 ώρες,Περίπου τις μισές διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Μένω μόνος/-η μου,< 10 λεπτά,"Αθλητισμό, Video Games, Σειρές / Ταινίες",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,20,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο.,0 - 2 ώρες,Όλες τις διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Μένω μόνος/-η μου,< 10 λεπτά,"Video Games, Εθελοντισμός",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
5,22,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο.,2 - 5 ώρες,Περίπου τις μισές διαλέξεις,Όχι,Διδακτορικές Σπουδές,Μένω μόνος/-η μου,25 - 45 λεπτά,"Σειρές / Ταινίες, Εθελοντισμός",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
6,19,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,0 - 2 ώρες,Περίπου τις μισές διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Συγκατοικώ με τα αδέρφια μου,10 - 25 λεπτά,"Αθλητισμό, Video Games",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
7,20,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,0 - 2 ώρες,Όλες τις διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Ζω με τους γονείς μου (ή γενικότερα την οικογέ...,< 10 λεπτά,"Video Games, Ξένη γλώσσα, Σειρές / Ταινίες",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
8,20,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,> 5 ώρες,Όλες τις διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Μένω μόνος/-η μου,< 10 λεπτά,"Video Games, Σειρές / Ταινίες, Πανεπιστημιακές...",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
9,22,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο.,> 5 ώρες,Όλες τις διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Ζω με τους γονείς μου (ή γενικότερα την οικογέ...,10 - 25 λεπτά,"Αθλητισμό, Σειρές / Ταινίες",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [3]:
data.describe()

,Ηλικία,NCO-01-01,NCO-01-02,NCO-01-03,NCO-01-04,NCO-01-05,NCO-02-01,NCO-02-02,NCO-02-03,NCO-02-04,...,NDM-06-02,NDM-06-04,NIS-07-08,NDM-07-01,NDM-07-02,NDM-08-02,NGE-08-02,NIS-07-06,NDM-08-03,NIS-08-04
count,39.000000,39.000000,39.000000,39.000000,39.000000,39.000000,39.000000,39.000000,39.000000,39.000000,...,39.000000,39.000000,39.000000,39.0,39.000000,39.0,39.000000,39.000000,39.000000,39.000000
mean,20.589744,6.256410,8.282051,6.487179,5.846154,6.846154,6.307692,6.705128,6.102564,6.576923,...,-0.615385,-0.743590,-0.871795,-1.0,-0.794872,-1.0,-0.794872,-0.717949,-0.794872,-0.794872
std,1.816850,2.807115,2.235464,3.227120,3.166755,3.717232,2.903300,2.310496,3.522819,3.396086,...,1.710884,1.312249,0.800641,0.0,1.281025,0.0,1.281025,1.761410,1.281025,1.281025
min,18.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000
25%,19.000000,5.000000,7.000000,5.000000,5.000000,5.500000,5.500000,6.000000,6.000000,5.000000,...,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000
50%,20.000000,6.000000,9.000000,7.000000,6.000000,8.000000,7.000000,7.000000,7.000000,8.000000,...,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000
75%,21.000000,8.000000,10.000000,8.500000,8.000000,10.000000,8.000000,8.000000,8.000000,9.000000,...,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000
max,27.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,...,8.000000,7.000000,4.000000,-1.0,7.000000,-1.0,7.000000,10.000000,7.000000,7.000000


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 97 columns):
Ηλικία                                                                 39 non-null int64
Φύλο                                                                   39 non-null object
Επέλεξα τη σχολή μου διότι:                                            39 non-null object
Κατά μέσο όρο την εβδομάδα, διαβάζω:                                   39 non-null object
Μέσα στο εξάμηνο, παρακαλουθώ:                                         39 non-null object
Υπήρξε ανάγκη για φροντηστηριακή βοήθεια σε κάποιο μάθημα έως τώρα;    39 non-null object
Μετά το πτυχίο, θα ήθελα να ακολουθήσω:                                39 non-null object
Ποιο από τα παρακάτω ισχύει;                                           39 non-null object
Η σχολή απέχει από το σπίτι μου:                                       39 non-null object
Ασχολούμαι εβδομαδιαία με:                                             39 non-null objec

In [5]:
from sklearn.model_selection import LeaveOneOut
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error,mean_squared_error
import math

import warnings #needed for this type of classifier
warnings.simplefilter(action='ignore', category=Warning)


In [6]:
# Let's drop categorical columns for now
# print(len(data.columns))
selected_columns = [0] + list(range(10,len(data.columns)))
print(selected_columns)

[0, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96]


In [7]:
# Select only those columns
data_selected = data.iloc[:,selected_columns]


In [8]:

sum_error = 0
sum_squared_error = 0
for i in range(1,len(data_selected.columns)): #For each course
    print(data_selected.columns[i])
    errors = []
    sq_errors = []
    X = data_selected.drop(data_selected.columns[i],axis=1,inplace=False)
    y = data_selected.iloc[:,i]
    loo = LeaveOneOut()
    xgb = XGBRegressor(objective = 'reg:squarederror')
    for train_index, test_index in loo.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]
        xgb.fit(X_train,y_train)
        predictions = xgb.predict(X_test)
        errors.append(mean_absolute_error(y_test,predictions))
        sq_errors.append(mean_squared_error(y_test,predictions))
    
    sum_error += np.mean(errors)
    sum_squared_error += np.mean(sq_errors)**(1/2)
    print("MAE:" + str(np.mean(errors)))
    print("RMSE:" + str((np.mean(sq_errors))**(1/2)))

print("Mean MAE:" + str( sum_error / (len(data_selected.columns) - 1)))
print("Mean RMSE:" + str( sum_squared_error / (len(data_selected.columns) - 1)))


NCO-01-01
MAE:1.772161439443246
RMSE:2.3777707593103576
NCO-01-02
MAE:1.3143336497820342
RMSE:2.0462830313028517
NCO-01-03
MAE:2.0043184023637037
RMSE:2.726573998829074
NCO-01-04
MAE:1.8158459327159784
RMSE:2.3484060626932095
NCO-01-05
MAE:2.288674553235372
RMSE:3.0424423119771236
NCO-02-01
MAE:2.4062163890936437
RMSE:3.1966515638058213
NCO-02-02
MAE:2.2061207707111654
RMSE:3.0602922541367334
NCO-02-03
MAE:2.405821048296415
RMSE:3.402372088402676
NCO-02-04
MAE:2.2608543596206565
RMSE:3.0331388934474908
NCO-02-05
MAE:1.6322617683655176
RMSE:2.215833056368948
NCO-03-01
MAE:2.5556486417085695
RMSE:3.3059490561096454
NCO-03-02
MAE:1.5907131357070727
RMSE:2.749645897790642
NCO-03-03
MAE:2.9692863722642264
RMSE:3.852726678495546
NCO-03-04
MAE:1.6411093045503666
RMSE:2.0367594189194707
NCO-03-05
MAE:1.9483456703332753
RMSE:3.0250059493632806
NCO-04-01
MAE:2.5020559269648333
RMSE:3.2236713860392965
NCO-04-02
MAE:1.224988497220553
RMSE:1.7346829864945879
NCO-04-03
MAE:1.772288889457018
RMSE:2.5